In [34]:
import numpy as np
import pandas as pd

def rbf_kernel(x1, x2, sigma):
  return np.exp(-np.linalg.norm(x1-x2)**2 / (2 * sigma**2))

def linear_kernel(x1, x2):
  return np.dot(x1, x2)

def multiquadric_kernel(x1, x2, sigma):
  return np.sqrt(np.linalg.norm(x1-x2)**2 + sigma**2)

def kmeans_clustering(data, k, iters):
  rng = np.random.default_rng(seed=0)
  m = data.shape[0]
  label = rng.choice(range(k), m) # initial random labels
  kmeans = np.zeros((k,data.shape[1])) # mean matrix, jth row is mean vector for jth cluster
  kstd = np.zeros((k,data.shape[1]))
  for iter in range(iters):
    for cluster in range(k):
      cluster_points = data[label==cluster]
      if cluster_points.size==0:
        continue
      kmeans[cluster] = np.mean(cluster_points, axis=0)
      kstd[cluster] = np.std(cluster_points, axis=0)
    
    # updating labels
    for i in range(m):
      distances = []
      for j in range(k):
        diff = data[i] - kmeans[j]
        dist = np.sum(np.square(diff))
        distances.append(dist)
      label[i] = np.argmin(distances)
  return kmeans,np.mean(kstd)

def holdout(data, trainp, validp):
  m = np.shape(data)[0]

  train = data[0 : int(np.floor(m*trainp/100))]
  valid = data[int(np.floor(m*trainp/100)) : int(np.floor(m*trainp/100))+int(np.floor(m*validp/100))]
  test = data[int(np.floor(m*trainp/100))+int(np.floor(m*validp/100)) : None]

  y_train = train[:,-1] #shape = (rows,)
  x_train = np.delete(train, -1, axis=1)

  y_valid = valid[:,-1] #shape = (rows,)
  x_valid = np.delete(valid, -1, axis=1)

  y_test = test[:,-1] #shape = (rows,)
  x_test = np.delete(test, -1, axis=1)

  return x_train,x_valid,x_test,y_train,y_valid,y_test

def five_fold(data):
  y = data[:,-1]
  x = np.delete(data, -1, axis=1)
  x_subsets = np.array_split(x, 5)
  y_subsets = np.array_split(y, 5)
  return x_subsets,y_subsets

def appendones(x):
  m = np.shape(x)[0]
  return np.concatenate((np.ones((m,1)),x), axis=1)

def one_hot_encode(y):
  len = np.size(y)
  encoded_y = np.zeros((len,3))
  for i in range(len):
    if y[i] == 1:
      encoded_y[i,0] = 1
    if y[i] == 2:
      encoded_y[i,1] = 1
    if y[i] == 3:
      encoded_y[i,2] = 1
  return encoded_y

def performance(y, pred):
  m = np.zeros((3,3)) # confusion matrix
  for p in range(len(pred)):
    if pred[p]==1 and y[p]==1:
      m[0,0]+=1
    if pred[p]==2 and y[p]==2:
      m[1,1]+=1
    if pred[p]==3 and y[p]==3:
      m[2,2]+=1
    if pred[p]==1 and y[p]==2:
      m[1,0]+=1
    if pred[p]==1 and y[p]==3:
      m[2,0]+=1
    if pred[p]==2 and y[p]==1:
      m[0,1]+=1
    if pred[p]==2 and y[p]==3:
      m[2,1]+=1
    if pred[p]==3 and y[p]==1:
      m[0,2]+=1
    if pred[p]==3 and y[p]==2:
      m[1,2]+=1
  ind_accuracy = [m[0,0]/np.sum(m[0,:]), m[1,1]/np.sum(m[1,:]), m[2,2]/np.sum(m[2,:])]
  accuracy = (m[0,0]+m[1,1]+m[2,2])/np.sum(m)
  return ind_accuracy, accuracy

def pred(y):
  return (np.argmax(y, axis=1) + 1)

In [35]:
data = pd.read_excel('/content/drive/MyDrive/NNFL Assignments (Aug 2021)/Assignment 2/data5.xlsx')
cols = np.array(data.columns, ndmin=2)
data = data.to_numpy()
data = np.concatenate((cols,data), axis=0)

# shuffle & train-test-valid-split
np.random.seed(0)
np.random.shuffle(data)
x_train, x_valid, x_test, y_train, y_valid, y_test = holdout(data, 70, 10)

# normalizing input data
mu = np.mean(x_train, axis=0)
std = np.std(x_train, axis=0)

x_train = (x_train-mu)/std
x_valid = (x_valid-mu)/std
x_test = (x_test-mu)/std

# one-hot-encoding output data
y_train_coded = one_hot_encode(y_train)
y_valid_coded = one_hot_encode(y_valid)
y_test_coded = one_hot_encode(y_test)

# appending ones (input for bias)
x_train = appendones(x_train)
x_valid = appendones(x_valid)
x_test = appendones(x_test)

m_train = np.size(y_train)
m_valid = np.size(y_valid)
m_test = np.size(y_test)

Using Gaussian Kernel

In [32]:
# parameters
k = 12
mu, sigma = kmeans_clustering(x_train, k, 200)

# training
y1 = np.zeros((m_train,k))
for i in range(m_train):
  y1[i] = np.array([rbf_kernel(x_train[i], mu[j], sigma) for j in range(k)])

w = np.linalg.pinv(y1) @ y_train_coded

# testing
y_11 = np.zeros((m_test,k))
for i in range(m_test):
  y_11[i] = np.array([rbf_kernel(x_test[i], mu[j], sigma) for j in range(k)])

y = y_11 @ w
y = pred(y)

# performance measures
ind_accuracy, accuracy = performance(y_test, y)
print("accuracy of class 1 = {}".format(ind_accuracy[0]))
print("accuracy of class 2 = {}".format(ind_accuracy[1]))
print("accuracy of class 3 = {}".format(ind_accuracy[2]))
print("overall accuracy of classifier = {}".format(accuracy))

accuracy of class 1 = 0.9
accuracy of class 2 = 1.0
accuracy of class 3 = 0.8421052631578947
overall accuracy of classifier = 0.9047619047619048


In [36]:
# using 5-fold crossvalidation
x_subsets, y_subsets = five_fold(data)

accuracy_vals = [] # accuracy from all folds
ind_accuracy1 = [] # class1 accuracy from all folds
ind_accuracy2 = [] # class2 accuracy from all folds
ind_accuracy3 = [] # class3 accuracy from all folds

# 5 fold CV
for fold in range(5):
  # test-train split
  x_test = x_subsets[fold]
  y_test = y_subsets[fold]

  x_train = np.concatenate(np.delete(x_subsets, fold, 0), axis=0)
  y_train = np.concatenate(np.delete(y_subsets, fold, 0), axis=0)

  # normalizing input data
  mu = np.mean(x_train, axis=0)
  std = np.std(x_train, axis=0)

  x_train = (x_train-mu)/std
  x_test = (x_test-mu)/std

  # one-hot-encoding output data
  y_train_coded = one_hot_encode(y_train)
  y_test_coded = one_hot_encode(y_test)

  # appending ones (input for bias)
  x_train = appendones(x_train)
  x_test = appendones(x_test)

  m_train = np.size(y_train)
  m_test = np.size(y_test)

  # parameters
  k = 12
  mu, sigma = kmeans_clustering(x_train, k, 200)

  # training
  y1 = np.zeros((m_train,k))
  for i in range(m_train):
    y1[i] = np.array([rbf_kernel(x_train[i], mu[j], sigma) for j in range(k)])

  w = np.linalg.pinv(y1) @ y_train_coded

  # testing
  y_11 = np.zeros((m_test,k))
  for i in range(m_test):
    y_11[i] = np.array([rbf_kernel(x_test[i], mu[j], sigma) for j in range(k)])

  y = y_11 @ w
  y = pred(y)

  # performance measures
  ind_accuracy, accuracy = performance(y_test, y)
  accuracy_vals.append(accuracy)
  ind_accuracy1.append(ind_accuracy[0])
  ind_accuracy2.append(ind_accuracy[1])
  ind_accuracy3.append(ind_accuracy[2])

print("mean accuracy of class 1 = {}".format(ind_accuracy[0]))
print("mean accuracy of class 2 = {}".format(ind_accuracy[1]))
print("mean accuracy of class 3 = {}".format(ind_accuracy[2]))
print("mean overall accuracy of classifier = {}".format(accuracy))

mean accuracy of class 1 = 0.7
mean accuracy of class 2 = 1.0
mean accuracy of class 3 = 0.8421052631578947
mean overall accuracy of classifier = 0.8571428571428571


Using Multiquadric Kernel

In [30]:
# parameters
k = 12
mu, sigma = kmeans_clustering(x_train, k, 200)

# training
y1 = np.zeros((m_train,k))
for i in range(m_train):
  y1[i] = np.array([multiquadric_kernel(x_train[i], mu[j], sigma) for j in range(k)])

w = np.linalg.pinv(y1) @ y_train_coded

# testing
y_11 = np.zeros((m_test,k))
for i in range(m_test):
  y_11[i] = np.array([multiquadric_kernel(x_test[i], mu[j], sigma) for j in range(k)])

y = y_11 @ w
y = pred(y)

# performance measures
ind_accuracy, accuracy = performance(y_test, y)
print("accuracy of class 1 = {}".format(ind_accuracy[0]))
print("accuracy of class 2 = {}".format(ind_accuracy[1]))
print("accuracy of class 3 = {}".format(ind_accuracy[2]))
print("overall accuracy of classifier = {}".format(accuracy))

accuracy of class 1 = 1.0
accuracy of class 2 = 1.0
accuracy of class 3 = 0.7894736842105263
overall accuracy of classifier = 0.9047619047619048


In [37]:
# using 5-fold crossvalidation
x_subsets, y_subsets = five_fold(data)

accuracy_vals = [] # accuracy from all folds
ind_accuracy1 = [] # class1 accuracy from all folds
ind_accuracy2 = [] # class2 accuracy from all folds
ind_accuracy3 = [] # class3 accuracy from all folds

# 5 fold CV
for fold in range(5):
  # test-train split
  x_test = x_subsets[fold]
  y_test = y_subsets[fold]

  x_train = np.concatenate(np.delete(x_subsets, fold, 0), axis=0)
  y_train = np.concatenate(np.delete(y_subsets, fold, 0), axis=0)

  # normalizing input data
  mu = np.mean(x_train, axis=0)
  std = np.std(x_train, axis=0)

  x_train = (x_train-mu)/std
  x_test = (x_test-mu)/std

  # one-hot-encoding output data
  y_train_coded = one_hot_encode(y_train)
  y_test_coded = one_hot_encode(y_test)

  # appending ones (input for bias)
  x_train = appendones(x_train)
  x_test = appendones(x_test)

  m_train = np.size(y_train)
  m_test = np.size(y_test)

  # parameters
  k = 12
  mu, sigma = kmeans_clustering(x_train, k, 200)

  # training
  y1 = np.zeros((m_train,k))
  for i in range(m_train):
    y1[i] = np.array([multiquadric_kernel(x_train[i], mu[j], sigma) for j in range(k)])

  w = np.linalg.pinv(y1) @ y_train_coded

  # testing
  y_11 = np.zeros((m_test,k))
  for i in range(m_test):
    y_11[i] = np.array([multiquadric_kernel(x_test[i], mu[j], sigma) for j in range(k)])

  y = y_11 @ w
  y = pred(y)

  # performance measures
  ind_accuracy, accuracy = performance(y_test, y)
  accuracy_vals.append(accuracy)
  ind_accuracy1.append(ind_accuracy[0])
  ind_accuracy2.append(ind_accuracy[1])
  ind_accuracy3.append(ind_accuracy[2])

print("mean accuracy of class 1 = {}".format(ind_accuracy[0]))
print("mean accuracy of class 2 = {}".format(ind_accuracy[1]))
print("mean accuracy of class 3 = {}".format(ind_accuracy[2]))
print("mean overall accuracy of classifier = {}".format(accuracy))

mean accuracy of class 1 = 1.0
mean accuracy of class 2 = 1.0
mean accuracy of class 3 = 0.8421052631578947
mean overall accuracy of classifier = 0.9285714285714286


Using Linear Kernel

In [31]:
# parameters
k = 12
mu, sigma = kmeans_clustering(x_train, k, 200)

# training
y1 = np.zeros((m_train,k))
for i in range(m_train):
  y1[i] = np.array([linear_kernel(x_train[i], mu[j]) for j in range(k)])

w = np.linalg.pinv(y1) @ y_train_coded

# testing
y_11 = np.zeros((m_test,k))
for i in range(m_test):
  y_11[i] = np.array([linear_kernel(x_test[i], mu[j]) for j in range(k)])

y = y_11 @ w
y = pred(y)

# performance measures
ind_accuracy, accuracy = performance(y_test, y)
print("accuracy of class 1 = {}".format(ind_accuracy[0]))
print("accuracy of class 2 = {}".format(ind_accuracy[1]))
print("accuracy of class 3 = {}".format(ind_accuracy[2]))
print("overall accuracy of classifier = {}".format(accuracy))

accuracy of class 1 = 0.9
accuracy of class 2 = 1.0
accuracy of class 3 = 0.8947368421052632
overall accuracy of classifier = 0.9285714285714286


In [38]:
# using 5-fold crossvalidation
x_subsets, y_subsets = five_fold(data)

accuracy_vals = [] # accuracy from all folds
ind_accuracy1 = [] # class1 accuracy from all folds
ind_accuracy2 = [] # class2 accuracy from all folds
ind_accuracy3 = [] # class3 accuracy from all folds

# 5 fold CV
for fold in range(5):
  # test-train split
  x_test = x_subsets[fold]
  y_test = y_subsets[fold]

  x_train = np.concatenate(np.delete(x_subsets, fold, 0), axis=0)
  y_train = np.concatenate(np.delete(y_subsets, fold, 0), axis=0)

  # normalizing input data
  mu = np.mean(x_train, axis=0)
  std = np.std(x_train, axis=0)

  x_train = (x_train-mu)/std
  x_test = (x_test-mu)/std

  # one-hot-encoding output data
  y_train_coded = one_hot_encode(y_train)
  y_test_coded = one_hot_encode(y_test)

  # appending ones (input for bias)
  x_train = appendones(x_train)
  x_test = appendones(x_test)

  m_train = np.size(y_train)
  m_test = np.size(y_test)

  # parameters
  k = 12
  mu, sigma = kmeans_clustering(x_train, k, 200)

  # training
  y1 = np.zeros((m_train,k))
  for i in range(m_train):
    y1[i] = np.array([linear_kernel(x_train[i], mu[j]) for j in range(k)])

  w = np.linalg.pinv(y1) @ y_train_coded

  # testing
  y_11 = np.zeros((m_test,k))
  for i in range(m_test):
    y_11[i] = np.array([linear_kernel(x_test[i], mu[j]) for j in range(k)])

  y = y_11 @ w
  y = pred(y)

  # performance measures
  ind_accuracy, accuracy = performance(y_test, y)
  accuracy_vals.append(accuracy)
  ind_accuracy1.append(ind_accuracy[0])
  ind_accuracy2.append(ind_accuracy[1])
  ind_accuracy3.append(ind_accuracy[2])

print("mean accuracy of class 1 = {}".format(ind_accuracy[0]))
print("mean accuracy of class 2 = {}".format(ind_accuracy[1]))
print("mean accuracy of class 3 = {}".format(ind_accuracy[2]))
print("mean overall accuracy of classifier = {}".format(accuracy))

mean accuracy of class 1 = 1.0
mean accuracy of class 2 = 1.0
mean accuracy of class 3 = 0.8947368421052632
mean overall accuracy of classifier = 0.9523809523809523
